In [ ]:
import pandas as pd
import numpy as np
models_df = \
    pd.DataFrame([[0.2] + [0.9] * 15,
                    [0.1] + [0.2] * 15,
                    [0.7] + [0.95] * 5 + [0.2] * 10],
                    columns=['prob']+[f"s{n}" for n in range(1,16)],
                    index=['daily_imaging','weekly_verification','trending'])
models_df

In [ ]:
print(models_df.loc['daily_imaging'])
print('--------------------')
print(models_df['prob'])

In [ ]:
def priors_by_model(has_session_offsets):
    priors_results = {}
    for key in models_df.index:
        assert len(has_session_offsets) == len(models_df.loc[key][1:])
        session_probs_of_imaging = \
            map(lambda tpl:tpl[1] if tpl[0] else (1.0 - tpl[1]), 
                zip(has_session_offsets, models_df.loc[key][1:]))

        from functools import reduce
        priors_results[key] = reduce(lambda p_l,p_r: p_l * p_r, session_probs_of_imaging)
    return priors_results

priors_by_model([True, True, True, True, False] + [False] * 10)

In [ ]:
def posteriors_by_model(has_session_offsets):
    priors_by_models = priors_by_model(has_session_offsets)
    posteriors_by_models = {key:priors_by_models[key] * models_df.loc[key][0] for key in models_df.index}
    sum_of_posteriors = sum(posteriors_by_models.values())
    for key in posteriors_by_models:
        posteriors_by_models[key] /= sum_of_posteriors
    return posteriors_by_models

posteriors_by_model([True, False, True, True, True] + [False, False, False, True, False] * 2)